# Concurrent Orchestration with Custom Aggregator

## Overview

This notebook demonstrates how to **override the default aggregator** in concurrent workflows with a custom async callback function. Instead of simply collecting ChatMessages, we'll use an LLM to synthesize a concise, consolidated summary from multiple domain experts' outputs.

### Key Concepts:

1. **Custom Aggregator**: Override default with `.with_aggregator(callback)`
2. **LLM-based Synthesis**: Use chat client to consolidate expert outputs
3. **Flexible Output**: Return any type (string, dict, custom object)
4. **Result Processing**: Access `AgentExecutorResponse` list from all participants

### Architecture:

```
User Prompt
    ↓
[Internal Dispatcher]
   ↙     ↓      ↘
Researcher  Marketer  Legal
   ↘     ↓      ↙
[Custom Aggregator: LLM Synthesis]
    ↓
Consolidated Summary (string)
```

### Default vs. Custom Aggregator:

| Feature | Default Aggregator | Custom Aggregator |
|---------|-------------------|------------------|
| **Output Type** | `list[ChatMessage]` | Any type you define |
| **Processing** | Simple concatenation | Custom logic (e.g., LLM synthesis) |
| **Use Case** | Raw agent responses | Summarization, scoring, selection |

## Prerequisites

- Azure OpenAI configured with environment variables
- Azure CLI authentication: Run `az login`
- Agent Framework installed

## Setup and Imports

In [ ]:
import asyncio
from typing import Any

from agent_framework import ChatMessage, ConcurrentBuilder, Role
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('../../.env')


## Define Custom Aggregator Function

The aggregator callback receives `list[AgentExecutorResponse]` and returns any type.

### Implementation Steps:
1. Extract final assistant message from each agent
2. Format as sections for the summarizer LLM
3. Call LLM with consolidation instructions
4. Return synthesized summary as string

In [ ]:
async def run_concurrent_with_custom_aggregator() -> None:
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    deployment_name = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
    chat_client = AzureOpenAIChatClient(
        deployment_name=deployment_name,
        endpoint=endpoint,
        credential=AzureCliCredential()
    )

    researcher = chat_client.create_agent(
        instructions=(
            "You're an expert market and product researcher. Given a prompt, provide concise, factual insights,"
            " opportunities, and risks."
        ),
        name="researcher",
    )
    marketer = chat_client.create_agent(
        instructions=(
            "You're a creative marketing strategist. Craft compelling value propositions and target messaging"
            " aligned to the prompt."
        ),
        name="marketer",
    )
    legal = chat_client.create_agent(
        instructions=(
            "You're a cautious legal/compliance reviewer. Highlight constraints, disclaimers, and policy concerns"
            " based on the prompt."
        ),
        name="legal",
    )

    # Define custom aggregator callback that uses the chat client to summarize
    async def summarize_results(results: list[Any]) -> str:
        """Custom aggregator that synthesizes expert outputs using an LLM."""
        # Extract one final assistant message per agent
        expert_sections: list[str] = []
        for r in results:
            try:
                messages = getattr(r.agent_run_response, "messages", [])
                final_text = messages[-1].text if messages and hasattr(messages[-1], "text") else "(no content)"
                expert_sections.append(f"{getattr(r, 'executor_id', 'expert')}:\n{final_text}")
            except Exception as e:
                expert_sections.append(f"{getattr(r, 'executor_id', 'expert')}: (error: {type(e).__name__}: {e})")

        # Ask the model to synthesize a concise summary of the experts' outputs
        system_msg = ChatMessage(
            Role.SYSTEM,
            text=(
                "You are a helpful assistant that consolidates multiple domain expert outputs "
                "into one cohesive, concise summary with clear takeaways. Keep it under 200 words."
            ),
        )
        user_msg = ChatMessage(Role.USER, text="\n\n".join(expert_sections))

        response = await chat_client.get_response([system_msg, user_msg])
        # Return the model's final assistant text as the completion result
        return response.messages[-1].text if response.messages else ""

## Build Workflow with Custom Aggregator

### Key Method:
- **`.with_aggregator(callback)`**: Replaces default aggregator
- Callback signature: `async def(list[AgentExecutorResponse]) -> Any`
- Can be sync or async function

In [ ]:
    # Build with custom aggregator callback
    workflow = (
        ConcurrentBuilder()
        .participants([researcher, marketer, legal])
        .with_aggregator(summarize_results)
        .build()
    )

## Execute and Display Results

In [ ]:
    events = await workflow.run("We are launching a new budget-friendly electric bike for urban commuters.")
    outputs = events.get_outputs()

    if outputs:
        print("===== Final Consolidated Output =====")
        print(outputs[0])  # Get the first (and typically only) output

## Run the Complete Workflow

In [ ]:
await run_concurrent_with_custom_aggregator()

## Expected Output

```
===== Final Consolidated Output =====
Urban e-bike demand is rising rapidly due to eco-awareness, urban congestion, and high fuel costs,
with market growth projected at a ~10% CAGR through 2030. Key customer concerns are affordability,
easy maintenance, convenient charging, compact design, and theft protection. Differentiation opportunities
include integrating smart features (GPS, app connectivity), offering subscription or leasing options, and
developing portable, space-saving designs. Partnering with local governments and bike shops can boost visibility.

Risks include price wars eroding margins, regulatory hurdles, battery quality concerns, and heightened expectations
for after-sales support. Accurate, substantiated product claims and transparent marketing (with range disclaimers)
are essential. All e-bikes must comply with local and federal regulations on speed, wattage, safety certification,
and labeling. Clear warranty, safety instructions (especially regarding batteries), and inclusive, accessible
marketing are required. For connected features, data privacy policies and user consents are mandatory.

Effective messaging should target young professionals, students, eco-conscious commuters, and first-time buyers,
emphasizing affordability, convenience, and sustainability. Slogan suggestion: "Charge Ahead—City Commutes Made
Affordable." Legal review in each target market, compliance vetting, and robust customer support policies are
critical before launch.
```

## Key Takeaways

### 1. Custom Aggregator Benefits
- **Synthesis**: Consolidate multiple perspectives into unified output
- **Filtering**: Select best response or combine intelligently
- **Scoring**: Rank or evaluate agent outputs
- **Formatting**: Transform to specific output structure

### 2. Aggregator Callback Signature
```python
async def custom_aggregator(results: list[AgentExecutorResponse]) -> Any:
    # results: List of responses from all participants
    # Return: Any type (string, dict, custom object)
    pass
```

### 3. Accessing Agent Responses
Each `AgentExecutorResponse` contains:
- `executor_id`: Identifier of the agent
- `agent_run_response`: The AgentRunResponse with messages
- `full_conversation`: Complete conversation history

### 4. LLM-Based Aggregation Pattern
1. Extract content from all agent responses
2. Format as structured input for summarizer
3. Use LLM to consolidate/synthesize
4. Return processed result

### 5. Other Custom Aggregator Examples

#### Best Response Selection:
```python
async def select_best(results: list[Any]) -> str:
    # Use LLM to evaluate and select best response
    evaluations = []
    for r in results:
        score = await evaluate_quality(r)
        evaluations.append((score, r))
    return max(evaluations, key=lambda x: x[0])[1]
```

#### Structured Output:
```python
async def create_report(results: list[Any]) -> dict:
    return {
        "research": extract_research(results[0]),
        "marketing": extract_marketing(results[1]),
        "legal": extract_legal(results[2]),
        "summary": await synthesize(results)
    }
```

#### Consensus Building:
```python
async def build_consensus(results: list[Any]) -> str:
    # Extract common themes across all responses
    # Identify disagreements
    # Use LLM to synthesize consensus view
    pass
```

### 6. Sync vs. Async Aggregators
- **Async**: Required if calling APIs (e.g., LLM synthesis)
- **Sync**: Fine for simple transformations or filtering

### 7. Error Handling
- Always handle cases where agent responses may be incomplete
- Use try-except when extracting from responses
- Provide fallback values for missing data

### 8. Production Considerations
- Monitor aggregation LLM costs (additional API call)
- Add timeout handling for aggregator logic
- Cache aggregation results if repeatable
- Log input and output of aggregator for debugging
- Consider fallback to default aggregator on errors